# Засоби підготовки та налізу даних
## Лабораторна робота №2
### ФБ-22 Лаптєв Денис
#### Підготовка середовища

In [3]:
from os import listdir
from datetime import datetime
import urllib.request
import pandas as pd
#import seaborn as sns
#import matplotlib.pyplot as plt
print ('setup complete')

setup complete


#### Завантаження датасетів

In [17]:
def dwn (i, y1, y2):
    url='https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID='+str(i)+'&year1='+str(y1)+'&year2='+str(y2)+'&type=Mean'

    wp = urllib.request.urlopen(url)
    text = wp.read()

    now = datetime.now()
    date_and_time_time = now.strftime('%d-%m-%Y_%H-%M-%S')
    out = open('data/NOAA_ID'+str(i)+'_'+date_and_time_time+'.csv','wb') # 'data/' — dir name
    out.write(text)
    out.close()




for i in range(1,28): # y 29
    dwn(i, 2000, 2020)

KeyboardInterrupt: 

#### Зчитування файлів та об'єднання в один датафрейм

In [4]:
def csvs_to_frame(dir):
    file_names = listdir(dir)

    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'AreaID']
    dfs = []

    for name in file_names:
        df = pd.read_csv(dir+name, header = 1, names = headers)
        df['AreaID'] = int(name[7:-24])

        # data cleaning
        df = df.drop(df.loc[df['VHI'] == -1].index)
        df = df.drop(df.index[-1])
        df.at[0, 'Year'] =  df.at[0, 'Year'][9:]
        df['Year'] = df['Year'].astype(int)

        dfs.append(df)

    frame = pd.concat(dfs).drop_duplicates().reset_index(drop=True)
    return frame


df = csvs_to_frame('data/')

##### Зміна індексів регіонів

In [ ]:
indexes = {1: 22, 2: 24, 3: 23, 4: 25, 5: 3, 6: 4, 7: 8, 8: 19, 9: 20, 10: 21, 11: 9, 13: 10, 14: 11, 15: 12, 16: 13,
           17: 14, 18: 15, 19: 16, 21: 17, 22: 18, 24: 1, 25: 2, 26: 7, 27: 5, 28:6} # no 28
df.replace({'AreaID': indexes}, inplace=True)

#### Виведення

##### 1
Ряд VHI для області за вказаний рік, пошук екстремумів (min та max);

In [70]:

def fun_1 (AreaID, year):
    result_df = df[(df['Year'] == year) & (df['AreaID'] == AreaID)][['Year', 'Week', 'VHI']]
    print ('max:', result_df['VHI'].min(), '\nmin:', result_df['VHI'].max())
    #sns.lineplot(x=result_df['Week'], y=result_df['VHI'])
    #print (result_df)


fun_1(24, 2000)

max: 11.25 
min: 63.27


##### 2
Ряд VHI за вказаний діапазон років для вказаних областей; виявити роки, протягом яких екстремальні посухи торкнулися більше вказаного відсотка областей по Україні (20% областей - 5 областей з 25);

In [71]:
def fun_21(y1, y2, areas):
    result_df = df[(df['Year'] >= y1) & (df['Year'] <= y2) & (df['AreaID'].isin(areas))]
    #plt.figure(figsize=(14,6)); sns.lineplot(x=result_df['Year'], y=result_df['VHI'], hue=result_df['AreaID'])
    #print (result_df)

fun_21(2000, 2020, [5, 20])

In [16]:
def fun_22(perc):
    amount= len(df["AreaID"].unique()) * (perc / 100)
    #print (amount)
    df_drought = df[df['VHI'] <= 15][['VHI', 'Year', 'Week', 'AreaID']] # обираютсья з посухою
    df_drought = df_drought.drop_duplicates(subset=['Year', 'AreaID']) # мають однакову комбінацію року та території
    # достатньо лиш одного тижня з посухою аби зарахувати рік
    #print (df_drought)
    counts = df_drought['Year'].value_counts() # кількістів рядків за роком
    #print(counts)
    result_year = counts[counts > amount] # більше за зазначене
    print(result_year)

fun_22(15)

Year
2000    6
2007    5
Name: count, dtype: int64



##### 3
Аналогічно для помірних посух

In [72]:
def fun_3(amount):
    df_drought = df[df['VHI'] <= 35][['VHI', 'Year', 'Week', 'AreaID']]
    df_drought = df_drought.drop_duplicates(subset=['Year', 'AreaID'])
    #print (df_drought)
    counts = df_drought['Year'].value_counts()
    result_year = counts[counts > amount]
    print(result_year)


fun_3(20)

Year
2000    27
2019    27
2015    24
Name: count, dtype: int64
